In [1]:
import pydub # faster than librosa
import numpy      as np
import pandas     as pd
import time

from pytorch.coughvid_dataset import CoughvidDataset
from pytorch.test_coughvid_dataset import TestCoughvidDataset
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models import resnet50

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy


#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter()

c:\python39\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
dir = 'C:/COUGHVID_public_dataset/public_dataset/'

full_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True)

dataframe = full_dataset.dataframe

minority_class_count = len(dataframe[dataframe['status']==2])

print(f'{minority_class_count} samples in the minority class.')

Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
8997 records ready to load across 2 groups.
699 samples in the minority class.


c:\python39\lib\site-packages\torchaudio\functional\functional.py:594: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


In [3]:
sample_dataset = CoughvidDataset(dir, 'metadata_compiled.csv', get_features=True, samples_per_class=100)


Mask file C:/COUGHVID_public_dataset/public_dataset/ does not exist. Calculating masks on the fly.
200 records ready to load across 2 groups.


In [4]:
# split data into training and test samples
train_indices, test_indices = train_test_split(np.arange(0,len(sample_dataset)-1), test_size=0.25)

batch_size  = 1
num_workers = 2

train_loader  = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(train_indices)
                          )

test_loader   = DataLoader(sample_dataset, 
                           num_workers=num_workers,
                           sampler=SubsetRandomSampler(test_indices)
                          )

dataloaders = {
    "train": train_loader,
    "test": test_loader
}

In [5]:
# load model and change output shape for binary prediction
model = resnet50()

model.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=2048,
        out_features=1
    ),
    torch.nn.Sigmoid()
)

model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)

model.double()


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
'''RESNET training code adapted from https://www.kaggle.com/gxkok21/resnet50-with-pytorch'''

#if torch.cuda.is_available():
#    model = model.cuda()
#    print('Using GPU.')

optimizer = torch.optim.Adam(model.parameters() )#, lr=10e-6)
criterion = torch.nn.BCELoss()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

EPOCHS = 2

for i in range(EPOCHS):
    for phase in ["train", "test"]:
        if phase == "train":
            model.train()
        else:
            model.eval()
        
        samples = 0
        loss_sum = 0
        correct_sum = 0
        for j, batch in enumerate(dataloaders[phase]):
            X,labels = batch
            #print(labels)
            #print(len(X),len(X[0]),len(X[1]))
            #print(labels[None,...].double())
            #X = torch.Tensor(X)
            #print(X.shape)
            if torch.cuda.is_available():
                X = X.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                y = model(X[None,...].double())
                #print(y,labels)
                loss = criterion(
                    y, 
                    labels[None,...].double()
                )
                #print(loss.item())
                #print(labels[None,...].double())
                
                #writer.add_scalar("Loss/train", loss, i)

                if phase == "train":
                    loss.backward()
                    optimizer.step()
                    
                loss_sum += loss.item() * X.shape[0] # We need to multiple by batch size as loss is the mean loss of the samples in the batch
                samples += X.shape[0]
                num_corrects = torch.sum((y >= 0.5).float() == labels[0].float())
                correct_sum += num_corrects
                
                # Print batch statistics every 50 batches
                if j % 50 == 49 and phase == "train":
                    print("{}:{} - loss: {}, acc: {}".format(
                        i + 1, 
                        j + 1, 
                        float(loss_sum) / float(samples), 
                        float(correct_sum) / float(samples)
                    ))
                
        # Print epoch statistics
        epoch_acc = float(correct_sum) / float(samples)
        epoch_loss = float(loss_sum) / float(samples)
        print("epoch: {} - {} loss: {}, {} acc: {}".format(i + 1, phase, epoch_loss, phase, epoch_acc))
        
        # Deep copy the model
        if phase == "test" and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "resnet50.pth")

C:\Users\Filip\AppData\Roaming\Python\Python39\site-packages\torch\cuda\__init__.py:80: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


1:50 - loss: 1.2700750309319708, acc: 0.54
1:100 - loss: 1.1651233135739592, acc: 0.56
epoch: 1 - train loss: 1.1585399742663707, train acc: 0.5234899328859061
epoch: 1 - test loss: 0.8681433698877377, test acc: 0.42
2:50 - loss: 0.9622638546150433, acc: 0.48
2:100 - loss: 0.9257594157908052, acc: 0.5
epoch: 2 - train loss: 0.8815286213228584, train acc: 0.4966442953020134
epoch: 2 - test loss: 0.7671184900417942, test acc: 0.42
